### Imports

In [82]:
import torch.nn as nn
import torch as torch
import pandas as pd
import cv2
from tqdm import tqdm
import os

### Model Class Definition and Utility Functions

In [83]:
# Softmax image classifier class
class SoftmaxClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SoftmaxClassifier, self).__init__()
        
        # Single fully connected layer
        self.linear = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        
        # Compute raw logits
        logits = self.linear(x)
        return logits

# Function to get the input dimension of a colored image
def get_input_dim(image_path):
    img_array = cv2.imread(image_path)
    flattened_array = img_array.reshape(-1)
    return len(flattened_array)

# Function to flatten a colored image into a 1D array
def flatten_image(image_path):
    img_array = cv2.imread(image_path)
    img_array = cv2.resize(img_array, (426, 320))
    flattened_array = img_array.reshape(-1)
    return flattened_array

# Count total files in directory and subdirectories
def count_files(directory):
    total = 0

    for root, dirs, files in os.walk(directory):
        total += len(files)
    return total

### Miscellaneous Testing Functions

### Configuration

In [84]:
# Model configuration
init_image_path = "/Users/ericcui/repos/imagenette/imagenette2-320/train/n01440764/ILSVRC2012_val_00000293.JPEG"
num_classes = 10
num_epochs = 10

# CSV processing configuration
train_image_label_csv = "/Users/ericcui/repos/imagenette/imagenette2-320/train_imagenette.csv"
path_prefix = "/Users/ericcui/repos/imagenette/imagenette2-320/"

# Label to index mappings
label_index_mappings = {
    "n01440764": 0,
    "n02102040": 1,
    "n02979186": 2,
    "n03000684": 3,
    "n03028079": 4,
    "n03394916": 5,
    "n03417042": 6,
    "n03425413": 7,
    "n03445777": 8,
    "n03888257": 9,
}

# Label to class mappings
class_mappings = {
    "n01440764": "tench",
    "n02102040": "English springer",
    "n02979186": "cassette player",
    "n03000684": "chain saw",
    "n03028079": "church",
    "n03394916": "French horn",
    "n03417042": "garbage truck",
    "n03425413": "gas pump",
    "n03445777": "golf ball",
    "n03888257": "parachute"
}

In [ ]:
# Function to resize an image to 426x320
def resize_image(image_path):
    # Read image
    img = cv2.imread(image_path)
    # Resize to 426x320
    resized_img = cv2.resize(img, (426, 320))
    return resized_img


# Using OpenCV (cv2)
def save_as_jpeg_cv2(numpy_array, output_path):
    """
    Save a numpy array as a JPEG image.
    
    Args:
        numpy_array: NumPy array of image (height, width, 3) in BGR format
        output_path: String path where to save the image (e.g., 'output.jpg')
    """
    success = cv2.imwrite(output_path, numpy_array)
    if success:
        print(f"Image successfully saved to {output_path}")
    else:
        print("Failed to save image")

### Training Hyperparameters

In [ ]:
# Hyperparameters
base_learning_rate = 0.001
batch_size = 32

### Prototyping

In [89]:
# Initialize the softmax classifier
input_dim = get_input_dim(init_image_path)
softmax_classifier = SoftmaxClassifier(input_dim, num_classes)

# Initialize the optimizer
optimizer = torch.optim.Adam(softmax_classifier.parameters(), lr=base_learning_rate)

# Read the CSV file containing image paths and labels
df = pd.read_csv(train_image_label_csv)

In [90]:
# Function to convert images from image paths to flattened tensors in a batch dataframe
def process_batch_images(batch_df):
    images = batch_df['path'].tolist()
    images = [path_prefix + path for path in images]
    images = [flatten_image(path) for path in images]
    
    images = [torch.tensor(img, dtype=torch.float32) for img in images]
    return images

In [87]:
batch_df = df.iloc[0:32]
images = process_batch_images(batch_df)
labels = batch_df['noisy_labels_0'].tolist()[:32]
# Convert string labels to indices using labels_index_mappings
labels = [label_index_mappings[label] for label in labels]


# Forward pass
outputs = softmax_classifier(torch.stack(images))


loss = nn.functional.cross_entropy(outputs, torch.tensor(labels))

# Backward pass and optimization
optimizer.zero_grad()
loss.backward()
optimizer.step()

print(f"Loss: {loss.item():.4f}")

Loss: 135.4510


In [ ]:
for i in tqdm(range(0, len(df), batch_size), desc="Training Progress"):
        # Get batch data
        batch_df = df.iloc[i:i+batch_size]
        
        # Convert images and labels
        images = process_batch_images(batch_df)
        labels = batch_df['noisy_labels_0'].tolist()[i:i+batch_size]
        labels = [label_index_mappings[label] for label in labels]
        
        # Forward pass
        outputs = softmax_classifier(torch.stack(images))
        loss = nn.functional.cross_entropy(outputs, torch.tensor(labels))
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
# Training loop
num_epochs = 10
batch_size = 32

# Track losses for plotting
train_losses = []

for epoch in range(num_epochs):
    epoch_loss = 0
    num_batches = 0
    
    


In [88]:
print(loss.grad_fn)

In [125]:
for epoch in range(num_epochs):
    for i in range(0, len(df), batch_size):
        
        # Getting the dataframe for the batch
        batch_df = df.iloc[i:i+batch_size]
        
        # Converting images to flattened tensors and labels to list
        images = process_batch_images(batch_df)
        labels = batch_df['noisy_labels_0'].tolist()
        
        
        

KeyboardInterrupt: 